# *Soumith Chintala, one of the co-authors of the DCGAN paper, made a presentation at NIPS 2016 titled How to Train a GAN?*
His recommendations are :

In [5]:
'''
1   Using a Gaussian latent space as input for the generator.
    This Gaussian distribution is a normal distribution of mean = 0 and std = 1
    such a random distribution is generate using numpy.random.randn()

    damn numpy really is something

    the shape of (500, 100) : there are 500 samples and each hasd 100 values, all of them are random,
    but centered around 0 with the std dev of 1.
'''

from numpy.random import randn

def latent_space(n_dims, n_samples):

    x = randn(n_dims * n_samples)
    x = x.reshape((n_samples, n_dims))

    return x

n_dims = 100
n_samples = 500

samples = latent_space(100, 500)

print(f'Space shape : {samples.shape}, Mean : {samples.mean()}, SD {samples.std()}')

Space shape : (500, 100), Mean : 0.007205884781732588, SD 0.9979680296881249


In [ ]:
'''
2   Training the Discriminator on real and fake batches separately rather than combining the two into a single mixed batch.
    this requires two calls to the train_on_batch()
    one using real inputs
    and the other using fake ones (generated ones)
'''

X_real, y_real = None, None # assuming we assign these from the dataset
X_fake, y_fake = None, None # assuming we assign these using generator output with 'real' label

# D model training assuming we have defined a discriminator
discriminator.train_on_batch(X_real, y_real)
discriminator.train_on_batch(X_fake, y_fake)

In [13]:
'''
3   Use smooth labels instead of hard/crisp labels
    instead of label 1 for real and 0 for fake,
    use values close to 1 for real and close to 0 for fake

    this has a regularization effect when training the model

    the numpy.random.random() returns random floats in the half-open interval [0.0, 1.0)
    which is useful for this purpose

    numpy.ones() returns a new array of given shape and type, filled with ones,
    numpy.zeroes() is the same for zeroes. both are useful here.

    some recommendations say only the positive smoothing (for real = 1) is needed
    but both can be applied
'''

from numpy.random import random
from numpy import ones
from numpy import zeros

def smooth_positive(y_pos):
    return y_pos - 0.3 + (random(y_pos.shape) * 0.5)  # gives values in (0.7, 1.2)

y_pos = ones((100,1)) # array of 100 samples, each of single values, all 100 are 1

y_p = smooth_positive(y_pos)

print(y_p.shape, y_p.min(), y_p.max())

def smooth_negative(y_neg):
    return y_neg + 0.3 * random(y_neg.shape)

y_neg = zeros((100,1)) # array of 100 samples, each of single values, all 100 are 0

y_n = smooth_negative(y_neg)

print(y_n.shape, y_n.min(), y_n.max())  # gives values in (0, 0.3)

(100, 1) 0.7103455119233193 1.1998710364500296
(100, 1) 0.000386294781397456 0.299211645481458


In [15]:
'''
4   Using noisy labels for the discriminator.
    The labels used when training the discriminator model are always correct. This means that fake
    images are always labeled with class 0 and real images are always labeled with class 1. It is
    recommended to introduce some errors to these labels where some fake images are marked as
    real, and some real images are marked as fake.

    the numpy.random.choice() helps with this using its optional 'size' argument

    the result below indicates that out of 100 1s, 5 turned to 0s
    and vice versa for the 100 0s
'''

from numpy import ones
from numpy import zeros
from numpy.random import choice

def noisy_labels(y, p_flip):
    # number of indices in y to flip the values of
    n_flips = int(p_flip * y.shape[0])

    # indices to flip values in y
    flip_i = choice([i for i in range(y.shape[0])], size = n_flips)

    # flip values
    y[flip_i] = 1 - y[flip_i]

    return y

n_samples = 100
real = ones((n_samples, 1))

noisy_real = noisy_labels(real, 0.05) # flip with 5% probablity

print(noisy_real.sum())

fake = zeros((n_samples, 1))

noisy_fake = noisy_labels(fake, 0.05)

print(noisy_fake.sum())

95.0
5.0
